In [1]:
# -*- coding: utf-8 -*-

# Traitement des EAF avec BDLexique
- la phonétisation dans un fichier-phonetique.eaf
- les éléments de BDLexique et les balises complémentaires dans un fichier.xml

In [2]:
import time, sys, codecs, re, glob
import pdb # ajouter pdb.set_trace() à l'endroit où on veut le débugueur
from lxml import etree as ET
import bs4
# import xml.etree.cElementTree as ET
import os, fnmatch, unidecode

# Préparatifs

In [3]:
annee=22

### Créer un parser XML

In [4]:
parser = ET.XMLParser(remove_blank_text=True)
debug=False

### Lire BDLexique

In [5]:
fichierLexique="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
fichier_exceptions=True

lexicon=codecs.open(fichierLexique,"r",encoding='utf8')
lBdlexique=lexicon.readlines()
lexicon.close()

In [6]:
bdlexique={}
for line in lBdlexique:
    line=line.strip()
    p=line.split(u';')
    if p[2]=="@" and not p[3] in ["N","V","J","K"]:
        p[1]+=p[2]
        p[2]=""
        if len(p)<7:
            for i in range(len(p)+1,7):
                p.append("")
    bdlexique[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8],p[9])

In [7]:
connecteurs=[
    u"et", u"alors", u"du coup", u"sinon", u"par contre", u"ça veut dire", u"enfin",
u"après", u"donc", u"puisque", u"puisqu'", u"en fait", u"mais", u"parce que", u"parce qu'", u"même si" , u"d'abord", u"et puis"
]
motTheme=[u"cible",u"frontières",u"accueillir",u"démonstrations",
          u"accompagner",u"contre-courant",u"importantes",u"droit de bouger",
          u"peur",u"solution",u"durable",u"refuge",u"assassins",u"tueurs",u"violeurs",
          u"fuient",u"misère",u"chaos",u"circulation",u"liberté",u"installation",
          u"solidarité",u"migrants",u"ouverture",u"frontières",u"conditions",u"réflexe",
          u"normal",u"sain",u"accueillir",u"moyens",u"intégrés",u"flux",u"migratoires",
          u"énormes",u"moyens",u"structures",u"loger",u"répondre",u"peuples",u"coopération",
          u"indéniable",u"réfugiés",u"digne",u"prendre la main",u"équilibrée",u"accueil",
          u"correctement",u"favorable",u"hébergés",u"mise à l’abri",u"centre humanitaire",
          u"situation",u"convenable",u"heurtée",u"valeurs",u"république",u"comportement",
          u"politique",u"associations",u"hébergement",u"respect",u"lien",u"confiance",
          u"travailleur",u"social",u"policiers",u"gazent",u"détruisent",u"nourriture",
          u"refusent",u"points d’eau",u"négation",u"partent",u"parcours",u"migration",
          u"terribles",u"touche",u"cimetière",u"choquant",u"scandaleux",u"dignité",
          u"impression",u"responsabilités",u"multiplication",u"conflits",u"excessivement",
          u"violents",u"solution",u"seule",u"quitter",u"victimes",u"esclavage",
          u"regression",u"humanité",u"guerre",u"état",u"pauvreté",u"pays",u"conditions",
          u"respect",u"organisme",u"ficher",u"contrôler",u"demande",u"papiers",u"droits",
          u"échec",u"aide",u"développement",u"vivre",u"famille",u"terrain",u"voir",
          u"souffrance",u"regression",u"France",u"richesse",u"humanité",u"extraordinaire",
          u"intérger",u"asile",u"question",u"temporaire",u"débat",u"souhaitons",
          u"propositions",u"européen",u"rapprocher",u"proches",u"position",u"absurdité",
          u"ONG",u"aider",u"détresse",u"lois",u"règles",u"principes",u"humanité",
          u"vulnérable",u"enfants",u"sauver",u"vies",u"générosité",u"classes",
          u"francophones",u"découverte",u"langue",u"logements",u"associations",
          u"continuerai",u"nouveaux",u"clé",u"chômage",u"héberger",u"employer",u"former",
          u"éduquer",u"milliers",u"vaincre",u"réticences",u"acccepter",u"aménage",
          u"refuges",u"régions",u"viennent",u"zones",u"protégées",u"normales",u"chez-eux",
          u"circuler",u"librement",u"Allemagne",u"obligations",u"humanitaire",u"préserver",
          u"populations",u"bataille",u"hégémonique",u"bataille",u"s'installer",u"mois",
          u"désaccord",u"moins",u"accueillant",u"bataille",u"idéologique",u"pression",
          u"chercher",u"immigrés",u"de force",u"délogés",u"expulsés",u"xénophobie",
          u"racistes",u"recul",u"idées",u"argent",u"dramatique",u"problèmes",u"économiques",
          u"interventions",u"militaires",u"famine",u"massacres",u"répression",u"bombardé",
          u"manqué",u"devoir",u"indigne",u"déposer",u"demande d'asile",u"absurde",
          u"campements",u"regrette",u"dispositifs",u"pérennes",u"divergence",
          u"appréciation",u"reconstitution",u"campements",u"crise",u"emballement",
          u"otage",u"noyer",u"guerre",u"bombardement",u"morts",u"difficultés",u"blocage",
          u"maltraiter",u"seul",u"privation",u"absurde",u"illusion",u"mensonge",u"ghettos",
          u"français",u"divisés",u"meurent",u"fuient",u"devoirs",u"moraux",
          u"centres de rétention",u"protection",u"cause",u"guerre",u"lieu",u"couteux",
          u"charge",u"services",u"gratuits",u"système",u"gestion",u"patrimoine",
          u"nationale",u"importante",u"fossé",u"travail",u"difficulté",u"maîtrise",
          u"suspendre",u"shengen",u"arrêter",u"immigration",u"communautarisme",
          u"fondamentalisme",u"islamiste",u"gel",u"constructions",u"mosquées",u"islam",
          u"radicaux",u"vivier",u"radicalisation",u"soldats",u"djihadisme",u"terroristes",
          u"puissance",u"terreau",u"frappent",u"clandestine",u"massive",u"insécurité",
          u"terrifiante",u"forces de l'ordre",u"mesures législatives",u"suppression",
          u"enrayer",u"processus",u"récupérer",u"clandestins",u"absence",u"bateaux",
          u"en mer",u"ramener",u"facilite",u"réseaux",u"criminels",u"laisser",u"mourir",
          u"sauver",u"ramener",u"port d’origine",u"corrompus",u"passeurs",u"arme",
          u"catastrophique",u"occupation",u"illégale",u"inexpulsable",u"engrenage",
          u"procédures",u"demandeurs",u"déboutés",u"économique",u"raisons",u"traverser",
          u"sortis",u"territoire",u"arrivés",u"origine",u"critères",u"extrêmement",
          u"incitatif",u"centres",u"AME",u"CMU",u"ensemble",u"folle",u"coût",u"faramineux",
          u"malvenu",u"opposition",u"opposée",u"communautarisme",u"fondamentalisme",
          u"imposer",u"ennemi",u"contre",u"rejet",u"anti-démocratique",u"enfermemement",
          u"séparation",u"pourissement",u"étranger",u"drame",u"faillite",u"déloyale",
          u"ménage",
            ]

### Formatage divers

In [8]:
def time2TRS(time):
    return unicode(float(time)/1000)

### Gestion API

In [9]:
consonnes=['k"', '(kt)"', 'n"', 'p"', 'R"', '@t"', 't"', '-V', '+V', '@z"', 'z"']
voyelles=["H", "j", "w", "E", "a", "2", "9", "6", "@", "y", "u", "O", u"ò", "o", "e", u"è", u"ê", u"û", u"ô", "i"]

# traduire SAMPA-BDLex en API

def sampa2api(sampa):
    if isinstance(sampa,str):
        api=sampa.decode("utf8")
    else:
        api=sampa
    api=api.replace(u'n"',u'n') 
    api=api.replace(u't"',u't') 
    api=api.replace(u'z"',u'z') 
    api=api.replace(u'R"',u'ʁ') 
    api=api.replace(u'p"',u'p') 
    api=api.replace(u'S',u'ʃ') 
    api=api.replace(u'Z',u'ʒ')
    api=api.replace(u'N',u'ŋ')
    api=api.replace(u'J',u'ɲ')
    api=api.replace(u'r',u'ʁ') 
    api=api.replace(u'H',u'ɥ')
    api=api.replace(u'E',u'ɛ')
    api=api.replace(u'2',u'ø')
    api=api.replace(u'9',u'œ')
    api=api.replace(u'6',u'ə')
    api=api.replace(u'O',u'ɔ')
    api=api.replace(u'è',u'e')   
    api=api.replace(u'ò',u'o')    
    api=api.replace(u'â',u'ɑ̃')   
    api=api.replace(u'ê',u'ɛ̃')   
    api=api.replace(u'û',u'œ̃')  
    api=api.replace(u'ô',u'ɔ̃')       
    api=api.replace(u'@',u'ə')
#     api=api.replace(u'R',u'ʁ') 
    return api

## Gestion Liaison

### Liaison possible

algorithme

+ si le mot courant et le suivant ne sont pas dans lexicon, pas de liaison
+ si le mot a une consonne dans le champ de la voyelle de liaison, check1 est vrai
+ si le mot suivant commence par une voyelle, check2 est vrai

  si check1 et check2 sont vrais, il y a liaison

In [10]:
def formerLiaison(mot1,mot2,contexteLiaison=False):
    if mot1[1][1]:
        result=sampa2api(mot1[1][1])
        if contexteLiaison:
#             print mot1, 
#             print mot2
            if mot1[1][2] in consonnes and mot2[1][1][0] in voyelles:
                result+=sampa2api(mot1[1][2].strip("()"))
    else:
        result="***%s***"%mot1[1][0]
    return result

In [11]:
print formerLiaison(["text",("1","tEkst","3")],None,False)

tɛkst


### Liaison obligatoire

algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [12]:
def liaison_obligatoire(mot1,mot2):
    determinant=["d", "P"]
    nom=["N", "G", "M"]
    adjectif=["J", "G", "M"]
    pronompers=["P"]
    verbe=["V"]
    cat1=mot1[1][3]
    cat2=mot2[1][3]
#     print cat1,cat2

    if cat1 in determinant and cat2 in nom :
        return True

    elif cat1 in determinant and cat2 in adjectif :
        return True

    elif cat1 in adjectif and cat2 in nom :
        return True
    
    elif cat1 in pronompers and cat2 in verbe :
        return True

    elif cat1 in verbe and cat2 in pronompers :
        return True

    else:
        return False

Cas de figure possibles:

- DET + N
    * ri + N:   d'animal, 
    * di + N:   certains éléphants
    * rd + N:   les animaux
    * dd + N:   ces étés, cet été
    * dp + N:   ton anorak
    * rc + N:   aux armes
- DET + ADJ:
    * ri + ADJ:   d'énormes
    * di + ADJ:   plusieurs immenses
    * rd + ADJ:   les immenses
    * dd + ADJ:   cet immense
    * dp + ADJ:   son immense
    * rc + ADJ:   aux immenses
- PERS + V:
    * SS + V:   m'épate
- V + PRO PERS: 
    * V + SS:   vont-ils


algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

### Liaison facultative

In [13]:
# vérifier si la liaison est facultative
def liaison_facultative(mot1,mot2):
    #pdb.set_trace()
    nom=["N", "G", "M"]
    pluriel=["MP", "FP"]
    adjectif=["J", "G", "M"]
    verbe=["V"]
    pronompers=["P"]
    adverbe=["A"]
    preposition=["p"]
    cat1=mot1[1][3]
    cat2=mot2[1][3]
    
    if (cat1 in nom) and (mot1[1][4] in pluriel) and (cat2 in adjectif) : 
        return True

    elif (cat1 in verbe) and (cat2 not in pronompers):
        return True

    elif cat1 in adverbe :
        return True
    
    elif cat1 in preposition : 
        return True

    else :
        return False

Cas de figure possibles :

- N pl + ADJ: 
    * N + ADJ: monstres énormes 
    * G + ADJ: rivaux énormes
- VERBE + TOUT-SAUF-PRO-PERS:
    * V + N sont éléphants
    * V + G sommes abdicaires
    * V + V sommes assis
    * V + A sommes admirablement
    * V + p sommes autour de
    * V + di ont aucune
    * V + rc sommes au
- ADV + QQCH:
    * ADV + N vraiment abruti
    * ADV + G vraiment abandonné
    * ADV + V vraiment aimé
    * ADV + J vraiment étonnant
    * ADV + ss vraiment ils
    * ADV + A vraiment étonnamment
    * ADV + p vraiment attendu
    * ADV + di vraiment autre 
    * ADV + rc vraiment au
- PREP + QQCH:
    * PREP + N très amoureux
    * PREP + G très abandonné
    * PREP + V très aimé
    * PREP + J très étonnant
    * PREP + SS très ils
    * PREP + A très étonnamment
    * PREP + p très attendu
    * PREP + di très autre
    * PREP + rc très au


## Traitement

+ Partie 1
*chaque phrase est prise individuellement,
    * découpée en blocs,
        * qui sont chacuns trimés si ce sont des mots
        * s'il y a plusieurs mots dans le bloc, ils sont séparés
    + Partie 2
    * pour chaque couple de mots
        * si la liaison est possible,
            * et qu'elle est obligatoire, l'api avec la liaison est généré
            * et qu'elle est facultative,
                * si l'utilisateur l'a choisi, l'api avec la liaison est généré
                * sinon l'api sans la liaison est généré

        + Partie 3
        * si la liaison n'est pas possible,
            * si le mot est dans bdlex, l'api est généré
            * sinon le mot est laissé tel quel (il a déjà les étoiles)        

    * pour le dernier mot de la phrase, 
        * si le mot est dans bdlex, l'api est généré
        * sinon le mot est laissé tel quel (il a déjà les étoiles) 

+ Partie 4
* le message à l'utilisateur et la phrase en api est imprimée

## Préparer structures Tours

In [14]:
altEspace=ur"\s|_"
connecteurs=[
    u"et", u"alors", u"du coup", u"sinon", u"par contre", u"ça veut dire", u"enfin",
u"après", u"donc", u"puisque", u"puisqu'", u"en fait", u"mais", u"parce que", u"parce qu'", u"même si" , u"d'abord", u"et puis"
]
immigration=[u"cible",u"frontières",u"accueillir",u"démonstrations",u"accompagner",u"contre-courant",u"importantes",u"droit de bouger",u"peur",u"solution",u"durable",u"refuge",u"assassins",u"tueurs",u"violeurs",u"fuient",u"misère",u"chaos",u"circulation",u"liberté",u"installation",u"solidarité",u"migrants",u"ouverture",u"frontières",u"conditions",u"réflexe",u"normal",u"sain",u"accueillir",u"moyens",u"intégrés",u"flux",u"migratoires",u"énormes",u"moyens",u"structures",u"loger",u"répondre",u"peuples",u"coopération",u"indéniable",u"réfugiés",u"digne",u"prendre la main",u"équilibrée",u"accueil",u"correctement",u"favorable",u"hébergés",u"mise à l’abri",u"centre humanitaire",u"situation",u"convenable",u"heurtée",u"valeurs",u"république",u"comportement",u"politique",u"associations",u"hébergement",u"respect",u"lien",u"confiance",u"travailleur",u"social",u"policiers",u"gazent",u"détruisent",u"nourriture",u"refusent",u"points d’eau",u"négation",u"partent",u"parcours",u"migration",u"terribles",u"touche",u"cimetière",u"choquant",u"scandaleux",u"dignité",u"impression",u"responsabilités",u"multiplication",u"conflits",u"excessivement",u"violents",u"solution",u"seule",u"quitter",u"victimes",u"esclavage",u"regression",u"humanité",u"guerre",u"état",u"pauvreté",u"pays",u"conditions",u"respect",u"organisme",u"ficher",u"contrôler",u"demande",u"papiers",u"droits",u"échec",u"aide",u"développement",u"vivre",u"famille",u"terrain",u"voir",u"souffrance",u"regression",u"France",u"richesse",u"humanité",u"extraordinaire",u"intérger",u"asile",u"question",u"temporaire",u"débat",u"souhaitons",u"propositions",u"européen",u"rapprocher",u"proches",u"position",u"absurdité",u"ONG",u"aider",u"détresse",u"lois",u"règles",u"principes",u"humanité",u"vulnérable",u"enfants",u"sauver",u"vies",u"générosité",u"classes",u"francophones",u"découverte",u"langue",u"logements",u"associations",u"continuerai",u"nouveaux",u"clé",u"chômage",u"héberger",u"employer",u"former",u"éduquer",u"milliers",u"vaincre",u"réticences",u"acccepter",u"aménage",u"refuges",u"régions",u"viennent",u"zones",u"protégées",u"normales",u"chez-eux",u"circuler",u"librement",u"Allemagne",u"obligations",u"humanitaire",u"préserver",u"populations",u"bataille",u"hégémonique",u"bataille",u"s'installer",u"mois",u"désaccord",u"moins",u"accueillant",u"bataille",u"idéologique",u"pression",u"chercher",u"immigrés",u"de force",u"délogés",u"expulsés",u"xénophobie",u"racistes",u"recul",u"idées",u"argent",u"dramatique",u"problèmes",u"économiques",u"interventions",u"militaires",u"famine",u"massacres",u"répression",u"bombardé",u"manqué",u"devoir",u"indigne",u"déposer",u"demande d'asile",u"absurde",u"campements",u"regrette",u"dispositifs",u"pérennes",u"divergence",u"appréciation",u"reconstitution",u"campements",u"crise",u"emballement",u"otage",u"noyer",u"guerre",u"bombardement",u"morts",u"difficultés",u"blocage",u"maltraiter",u"seul",u"privation",u"absurde",u"illusion",u"mensonge",u"ghettos",u"français",u"divisés",u"meurent",u"fuient",u"devoirs",u"moraux",u"centres de rétention",u"protection",u"cause",u"guerre",u"lieu",u"couteux",u"charge",u"services",u"gratuits",u"système",u"gestion",u"patrimoine",u"nationale",u"importante",u"fossé",u"travail",u"difficulté",u"maîtrise",u"suspendre",u"shengen",u"arrêter",u"immigration",u"communautarisme",u"fondamentalisme",u"islamiste",u"gel",u"constructions",u"mosquées",u"islam",u"radicaux",u"vivier",u"radicalisation",u"soldats",u"djihadisme",u"terroristes",u"puissance",u"terreau",u"frappent",u"clandestine",u"massive",u"insécurité",u"terrifiante",u"forces de l'ordre",u"mesures législatives",u"suppression",u"enrayer",u"processus",u"récupérer",u"clandestins",u"absence",u"bateaux",u"en mer",u"ramener",u"facilite",u"réseaux",u"criminels",u"laisser",u"mourir",u"sauver",u"ramener",u"port d’origine",u"corrompus",u"passeurs",u"arme",u"catastrophique",u"occupation",u"illégale",u"inexpulsable",u"engrenage",u"procédures",u"demandeurs",u"déboutés",u"économique",u"raisons",u"traverser",u"sortis",u"territoire",u"arrivés",u"origine",u"critères",u"extrêmement",u"incitatif",u"centres",u"AME",u"CMU",u"ensemble",u"folle",u"coût",u"faramineux",u"malvenu",u"opposition",u"opposée",u"communautarisme",u"fondamentalisme",u"imposer",u"ennemi",u"contre",u"rejet",u"anti-démocratique",u"enfermemement",u"séparation",u"pourissement",u"étranger",u"drame",u"faillite",u"déloyale",u"ménage"
]

In [15]:
immigrationMots=sorted(set(immigration))

In [16]:
def compterVoyelles(chaine):
    result=0
    for element in chaine:
        if element in voyelles:
            result+=1
    return result

### Lister les groupes

In [17]:
def listerGroupes(repRacine):
    return glob.glob(repRacine+ur"*")

### Chercher les inconnus 

In [18]:
def dicterInconnus(lInconnus):
    result={}
    for line in lInconnus:
        line=line.strip()
        p=line.split(";")
        if len(p[1])!=0:
            if len(p)>8:
                print p
                for i in range(len(p)+1,7):
                    p.append("")
            result[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8])
    return result

In [19]:
def lireInconnus(repGroupe):
    result={}
    fInconnus=repGroupe+"/inconnus.txt"
    try:
        with codecs.open(fInconnus,"r",encoding="utf8") as inFile:
            lInconnus=inFile.readlines()
        result=dicterInconnus(lInconnus)
        print result
    except:
        print "pas d'inconnus.txt"
    return result

### Lister les EAFs du groupe

In [20]:
def listerEAFs(repGroupe):
    result=[f for f in glob.glob(repGroupe+ur"/*.eaf") if "phonetique" not in unidecode.unidecode(f.lower())]
    return result

### Lister les mots d'un EAF

In [21]:
def trimer(mot):
#     mot=mot.lower()
    for p in u',;.-?!“”‘’‛‟′″´˝"«»':      # Modifié le 12/01/20 pour gérer les deux points comme marqueur dans les mots
        mot=mot.replace(p, ' ')
    mot=mot.strip()
    return mot

In [22]:
def deparentheser(mot):
    forme=mot
    graphie=mot
    m=re.search(ur"\([^)]*\)",forme)
    while (m):
        forme=re.sub(ur"\(([^)]*)\)","\g<1>", forme)
        graphie=re.sub(ur"\(([\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?)\)","'", graphie)
        m=re.search(ur"\([^)]*\)",forme)
    forme=forme.lstrip(")").rstrip("(")
    forme=forme.replace(":","")
    if graphie.endswith("("):
        graphie=graphie[:-1]+"'"
    return(forme,graphie)
#    motsAbreges[mot]={"lexical":forme, "graphie":graphie}    
    

In [23]:
trimer("Nor(m)al")

u'Nor(m)al'

In [24]:
def tokenizerTour(tour,lexique):
    listeMots=[]
    listeTokens=[]
    elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ():]+|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", tour)
    elements=[x for x in elements if x!=u" "]
    mots=[x for x in elements if not x in u"-_.…,—–()\[\]\/#\"“”‘«»<>'’=~:" and not x in [u" ;",u" !",u" ?",u" :"]]
    for element in elements:
        if element in mots:
            mot = trimer(element)
            (forme,graphie)=deparentheser(mot)
            listeMots.append(graphie)
            if forme.lower() in lexique:
                listeTokens.append(lexique[forme.lower()])
            else:
                listeTokens.append(forme.lower())
        else:
            listeMots.append(element)
            listeTokens.append(element)
    return listeMots,listeTokens

In [25]:
def tokenizerTexte(texte):
    listeMots=set()
    elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ():]+|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", texte)
    elements=[x for x in elements if x!=u" "]
    mots=[x for x in elements if not x in u"-_.…,—–()\[\]\/#\"“”‘«»<>'’=~:" and not x in [u" ;",u" !",u" ?",u" :"]]
    for mot in mots:
        mot = trimer(mot)
        (forme,graphie)=deparentheser(mot)
        listeMots.add(forme.lower())
    return list(listeMots)

In [26]:
def listerMots(nomEAF):
    xmlEAF=ET.parse(nomEAF,parser)
    texteLignesEAF=[]
    for tier in xmlEAF.xpath("//TIER"):
        for annotation in tier.xpath("ANNOTATION/ALIGNABLE_ANNOTATION"):
            aValue=annotation.xpath("ANNOTATION_VALUE/text()")
            texteLignesEAF.append("\n".join(aValue))
    texteEAF="\n".join(texteLignesEAF)
    return tokenizerTexte(texteEAF)

In [27]:
def faireLexique(nomEAF,groupeInconnus):
    extraitBdlexique={}
    listeMots=listerMots(nomEAF)
    for forme in listeMots:
        if forme in groupeInconnus:
            extraitBdlexique[forme]=groupeInconnus[forme]
        elif forme in bdlexique:
            extraitBdlexique[forme]=bdlexique[forme]
        else:
            groupeInconnus[forme]=(forme,"","","","","","","","")
            extraitBdlexique[forme]=(forme,"***%s***"%forme,"","","","","","","")
    return extraitBdlexique,groupeInconnus

In [28]:
def dicterContent(content,lexique):
    lMots,lTokens=tokenizerTour(content,lexique)
    result=[[mot,lTokens[nMot]] for nMot,mot in enumerate(lMots)]
    return result

### Changement de convention d'annotation pour les disfluences
- 2021: 
    - [blablabla] => \<disfluence\>blablabla\</disfluence\>
- 2022: 
    - [rire] => \<rire/\>
    - {blablabla} => \<disfluence\>blablabla\</disfluence\>
 

In [29]:
def makeAttribs(chaine):
    result=chaine
    mAttribs=re.match(ur"(\w+) (((\w+)=[\"\']?[^\"\']+[\"\']?\s*)+)",chaine)
    if mAttribs:
        result=mAttribs.group(1)+" "+mAttribs.group(2)
    else:
        mType=re.match(ur"(\w+) (.*)",chaine)
        if mType:
            result=mType.group(1)+" type='%s'"%mType.group(2)
    return result 

def makeTagAttribs(chaine):
    resTag=unidecode.unidecode(chaine)
    resAttrib={}
    mAttribs=re.match(ur"(\w+) (((\w+)=[\"\']?[^\"\']+[\"\']?\s*)+)",chaine,re.U)
    if mAttribs:
#         print mAttribs.groups()
        resTag=unidecode.unidecode(mAttribs.group(1))
        if "'" not in mAttribs.group(2) and '"' not in mAttribs.group(2):
            resAttrib=re.findall(ur"(\w+)=(\S+)",mAttribs.group(2),re.U)
#             print resAttrib
            resAttrib={unidecode.unidecode(k):v for k,v in resAttrib}            
        else:
            resAttrib=re.findall(ur"(\w+)=[\"\']([^\"\']+)[\"\']",mAttribs.group(2),re.U)
#             print resAttrib
            resAttrib={unidecode.unidecode(k):v for k,v in resAttrib}
    else:
        mType=re.match(ur"(\w+) (.*)",chaine,re.U)
        if mType:
            resTag=unidecode.unidecode(mType.group(1))
            resAttrib={"type":mType.group(2)}
    return resTag,resAttrib 

In [30]:
makeAttribs(u"e t=acc p=fRãsE")

u'e t=acc p=fR\xe3sE'

In [31]:
def baliserTour(tour,lexique):    
    lMorceaux=re.split(ur"(\[[^]]*\])",tour)
#     print lMorceaux
    newTour=[]
    for lMorceau in lMorceaux:
        m1=re.match(ur"\[(.*)\]",lMorceau)
        m2=re.match(ur"\{(.*)\}",lMorceau)
        if m1:
            bContent=m1.group(1)
            if bContent.startswith(u"="):
                newTour.append(["[%s]"%bContent,"<%s>"%makeAttribs(bContent[1:])])
            elif bContent.startswith(u"/"):
                newTour.append(["[%s]"%bContent,"</%s>"%bContent[1:]])
            else:
                newTour.append(["[%s]"%bContent,"<%s/>"%makeAttribs(bContent)])
        elif m2:
            aContent=m2.group(1)
            newTour.append(["{","<disfluence>"])
            aContentDict=dicterContent(aContent,lexique)
            newTour.extend(aContentDict)
            newTour.append(["}","</disfluence>"])

        else:
            if lMorceau.strip()!="":
                lMorceauDict=dicterContent(lMorceau.strip(),lexique)
                newTour.extend(lMorceauDict)
    return newTour

In [33]:
def trouverMot2(nMot1,tourBalises):
    result=None
    if len(tourBalises[nMot1])>1:
#         print tourBalises[nMot1][1]
        if isinstance(tourBalises[nMot1][1],tuple):
            for nMot2,mot2 in enumerate(tourBalises[nMot1+1:]):
                if len(mot2)>1:
#                     print mot2[0],mot2[1]
                    if isinstance(mot2[1],tuple):
                        result=nMot1+1+nMot2
                        break
    return result

In [34]:
def faireTourPhon(tourBalises):
    result=[]
    for nMot1 in range(len(tourBalises)):
        nMot2=trouverMot2(nMot1,tourBalises)
        mot1=tourBalises[nMot1]
#         print mot1
        if nMot2:
            mot2=tourBalises[nMot2]
            contexteLiaison=liaison_obligatoire(mot1,mot2)
#             print nMot1,mot1[1],contexteLiaison
            result.append(formerLiaison(mot1,mot2,contexteLiaison))
        else:
#             print mot1
            if len(mot1)==2 and isinstance(mot1[1],tuple):
                result.append(sampa2api(mot1[1][1]))
            elif len(mot1)==2:
#                 print "len(mot1)==2", mot1
                result.append(mot1[1])
            else:
#                 print "len(mot1)!=2", mot1
                result.append(mot1[0])
    return "".join(result)

In [35]:
def transcrireTourEAF(tour,lexique):
    tourBalises=baliserTour(tour,lexique)
    tourEAF=faireTourPhon(tourBalises)
    return tourEAF

In [36]:
def transcrireEAF(nomEAF,subBdlexique):
    xmlEAF=ET.parse(nomEAF,parser)
    texteLignesEAF=[]
    for nAnnotation,annotation in enumerate(xmlEAF.xpath("//ANNOTATION_VALUE")):
        if annotation.text:
            tourEAF=transcrireTourEAF(annotation.text,subBdlexique)
            annotation.text=tourEAF
    return xmlEAF
    

In [37]:
def enchasseMot(balise,mot,graphie,phono,nMot,nTour):
    ortho=mot[0]
    cat=mot[3]
    if cat in [u"J",u"K"]:
        cat=u"Adj"
    ms=mot[4]
    vs=mot[5]
    lexeme=mot[6].upper()
    freq=mot[8]
    nbVoyelles=compterVoyelles(mot[1])
    if u" " in vs:
        vs=u""
    motAttributs={"cat":cat,"ms":ms,"vs":vs,"phon":phono,"nbsyll":str(nbVoyelles),"ortho":ortho, "lexeme":lexeme, "freq":freq, "id":"%05d%03d"%(nTour,nMot)}
    baliseMotBDL=ET.SubElement(balise,"motBDL",motAttributs)
    baliseMotBDL.text=graphie
    return nbVoyelles
    

def enchasseNonMot(balise,nonMot):
    lNonMot=ET.SubElement(balise, "punct")
    lNonMot.text=nonMot
    

In [38]:
mapBalises={"???":"incomprehensible"}

In [39]:
def enchasserTourTRS(tour,tourMots):
    nbVoyelles=0
    nbMots=0
    lConnecteurs=set()
    lThemeMots=set()
#     print tourMots
#     print tour

    nTour=int(tour.attrib["id"])
    stackBalises=[tour]
    for nMot1,mot1 in enumerate(tourMots):
#         print nMot1,mot1
        graphie=mot1[0]
        lexMot=mot1[1]
        if lexMot:
            orthoMot=lexMot[0]
        else:
            orthoMot=""
        if orthoMot in connecteurs:
            lConnecteurs.add(orthoMot)
        if orthoMot in lThemeMots:
            lThemeMots.add(orthoMot)
        if isinstance(lexMot,tuple):
            nbMots+=1
            nMot2=trouverMot2(nMot1,tourMots)
            phono="???"
            if nMot2:
                mot2=tourMots[nMot2]
                contexteLiaison=liaison_obligatoire(mot1,mot2)
#                 print nMot1,mot1[1],contexteLiaison
                phono=formerLiaison(mot1,mot2,contexteLiaison)
                
            nbVoyelles+=enchasseMot(stackBalises[-1],lexMot,graphie,phono,nMot1,nTour)
        else:
            m=re.match(ur"</.*>",lexMot)
            if m:
                stackBalises.pop(-1)
#                 print "pop",groupeEAF
#                 print tourMots
#                 print stackBalises[-1].tag
            else:
                m=re.match(ur"<(.*[^/])/?>",lexMot)
                if m:
                    locBalise=m.group(1)
#                     print "stackBalises",ET.tostring(stackBalises[-1]),locBalise
                    if locBalise in mapBalises:
                        locBalise=mapBalises[locBalise]
                    locTag,locAttribs=makeTagAttribs(locBalise)
                    newBalise=ET.SubElement(stackBalises[-1],locTag,locAttribs)
                    stackBalises.append(newBalise)
#                     print "push",stackBalises[-1].tag
                else:
                    enchasseNonMot(stackBalises[-1],lexMot)
    tour.attrib["nbSyll"]=str(nbVoyelles)
    tour.attrib["nbMots"]=str(nbMots)
    if lConnecteurs:
        tour.attrib["connecteurs"]=" ".join(lConnecteurs)
    if lThemeMots:
        tour.attrib["themeMots"]=" ".join(lThemeMots)

        

In [40]:
turn=ET.Element("Turn")
tour=ET.SubElement(turn,"tour",id="00001")
e1=ET.SubElement(tour,"mot")
e1.text="mot1"
nom1=ET.SubElement(tour,"nom")
nom1e1=ET.SubElement(nom1,"mot")
nom1e1.text="nom1"
e2=ET.SubElement(tour,"mot")
e2.text="mot2"
print ET.tostring(turn,pretty_print=True,encoding="utf8")

<Turn>
  <tour id="00001">
    <mot>mot1</mot>
    <nom>
      <mot>nom1</mot>
    </nom>
    <mot>mot2</mot>
  </tour>
</Turn>



In [41]:
def transcrireTRS(nomEAF,lexique):
    xmlEAF=ET.parse(nomEAF,parser)

    #################
    #
    # Récupération des timestamps
    #
    #################
    
    ts={}
    for timeOrder in xmlEAF.xpath("//TIME_ORDER/TIME_SLOT"):
        tsID=timeOrder.attrib["TIME_SLOT_ID"]
        tsTime=timeOrder.attrib["TIME_VALUE"]
        ts[tsID]=tsTime
    tiersTypes={}
    annotations={}
    turnTextes=[]

    #################
    #
    # Récupération des tours
    #
    #################
    
    for tier in xmlEAF.xpath("//TIER"):
        tierID=tier.attrib["TIER_ID"]
        tierType=tier.attrib["LINGUISTIC_TYPE_REF"]
        tiersTypes[tierID]=tierType
        for annotation in tier.xpath("ANNOTATION/ALIGNABLE_ANNOTATION"):
            aID=annotation.attrib["ANNOTATION_ID"]
            aTS1=annotation.attrib["TIME_SLOT_REF1"]
            aTS2=annotation.attrib["TIME_SLOT_REF2"]
            aBegin=time2TRS(ts[aTS1])
            aEnd=time2TRS(ts[aTS2])
            aValue=annotation.xpath("ANNOTATION_VALUE/text()")
            turnTextes.append("\n".join(aValue))
            turn=(tierID, aBegin, aEnd, aValue)
            annotations[aID]=turn
            

    #################
    #
    # Fabrication entête TRS
    #
    #################

    root=ET.Element("Trans")
    speakers=ET.SubElement(root, "Speakers")
    speakerID={}
    for nSpk,spk in enumerate(tiersTypes):
        ET.SubElement(speakers,"Speaker",id="spk%d"%(nSpk+1),name=spk)
        speakerID[spk]="spk%d"%(nSpk+1)

    episode=ET.SubElement(root, "Episode")
    section=ET.SubElement(episode, "Section")   
    tourId=0
    for k,v in sorted(annotations.iteritems(),key=lambda x: int(x[0].strip("a"))):
        #
        # créer la structure Turn avec speaker, startTime, endTime
        # et tour avec id
        #
        turn=ET.SubElement(section,"Turn",speaker=speakerID[v[0]],startTime=v[1],\
                                          endTime=v[2])
        tour=ET.SubElement(turn,"tour",id="%05d"%tourId)
        #
        # découper le texte de l'annotation
        #
        vTextes=baliserTour("\n".join(v[3]),lexique)
        if debug: print vTextes
        enchasserTourTRS(tour,vTextes)
        tourId+=1
    tree = ET.ElementTree(root)
    return tree

In [42]:
def ecrireInconnus(dInconnus,repGroupe):
    fInconnus=repGroupe+"/inconnus.txt"
    with codecs.open(fInconnus,"w",encoding="utf8") as outFile:
        for inconnu in dInconnus.values():
            outFile.write(";".join(inconnu)+"\n")

# Main

In [43]:
repRacine="/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-20%d/0-Groupes/"%annee
repGroupes=listerGroupes(repRacine)
print repGroupes

[u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/BernardCantin', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/ChavierChristophePigeot', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/CoiffardRioWilburn', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/GARYMARREL', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/Wang', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/AntonLaujac ', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/DosSantosLabbe', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/BegayLapeyreLapeyre', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/GonzalezTostain', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/AndrieuxAntoinetteBernede', u'/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/Du

In [44]:
debug=0
for repGroupe in repGroupes[:]:
    print
    print repGroupe
    groupeInconnus=lireInconnus(repGroupe)
    groupeEAFs=listerEAFs(repGroupe)
    for groupeEAF in groupeEAFs:
        print "groupeEAF",groupeEAF.split("/")[-1]
        subBdlexique,groupeInconnus=faireLexique(groupeEAF,groupeInconnus)
        newEAF=transcrireEAF(groupeEAF,subBdlexique)
        fNewEAF=groupeEAF.replace(".eaf","-phonetique.eaf")
        newEAF.write(fNewEAF, pretty_print=True, encoding='utf-8', xml_declaration=True)
        newTRS=transcrireTRS(groupeEAF,subBdlexique)
        nomXML=groupeEAF.replace(".eaf",".xml")
        if nomXML!=groupeEAF:
            newTRS.write(nomXML, pretty_print=True, encoding='utf-8', xml_declaration=True)
        else:
            print "pb de nom EAF",nomEAF
    ecrireInconnus(groupeInconnus,repGroupe)


/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/BernardCantin
{}
groupeEAF BernardCantintranscriptionortho2.eaf
groupeEAF BernardCantintranscriptionortho3.eaf
groupeEAF BernardCantintranscriptionortho4.eaf
groupeEAF BernardCantintranscriptionortho5.eaf


/Users/gilles/opt/anaconda3/envs/python2/lib/python2.7/site-packages/unidecode/__init__.py:46: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  _warn_if_not_unicode(string)


groupeEAF BernardCantintranscriptionortho6.eaf
groupeEAF invitation-à-boire-2.eaf
groupeEAF invitation à boire 1.eaf
groupeEAF anniversaire.eaf
groupeEAF repasentreamis.eaf
groupeEAF wklyon.eaf
groupeEAF BernardCantintranscriptionortho7.eaf
groupeEAF Clapi_Signal_repas___conversation_repas_francais_vrai_4f4b911396.eaf
groupeEAF transcription 1 - invitation à jouer (jeux vidéo).eaf
groupeEAF BernardCantin transcriptions orthographiques.eaf
groupeEAF conversation-tel-québec.eaf
groupeEAF BernardCantintranscriptionsortho1.eaf

/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/ChavierChristophePigeot
{}
groupeEAF partie charlotte elan .eaf
groupeEAF Enregistrement partie Jade .eaf
groupeEAF Enregistrement.eaf
groupeEAF Enregistrement partie Ines.eaf

/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2022/0-Groupes/CoiffardRioWilburn
{}
groupeEAF Roman.eaf
groupeEAF homophobiechezlesjeunes_phonetic.eaf
groupeEAF cacommenceaujourdhui_fin.eaf
groupeEAF homophobiec